In [ ]:
import xarray as xr
import numpy as np
import os

import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
from matplotlib import ticker

In [ ]:
import pprint

In [ ]:
from cdo import *

In [ ]:
cdo = Cdo(tempdir='/scratch/k/k202141/cdotmp')
cdo.cleanTempDir()
# use cdo.cleanTempDir() at some point if you run many operations

In [ ]:
sns.set_context('notebook')
sns.set_style('whitegrid')

## Setup

Interactive notebook for analysing single experiments and quickly comparing different experiments

## Evaluate a single experiment

Define the experiment you want to analyse. Provide
 - `exp_root` The experiment root directory
 - `exp_name` The experiment name
 - `target_plot_dir` Where to save the generated plots

In [ ]:
exp_root = '/work/ka1176/caroline/gitlab/icon-aes/experiments/'
exp_name = 'warm_bubble_cffi'
out_atm_3d = f'{exp_name}_atm_3d_ml_20080801T000000Z.nc'
out_atm_2d = f'{exp_name}_atm_2d_ml_20080801T000000Z.nc'
target_plot_dir = f'./analysis_plots/'

In [ ]:
if not os.path.exists(target_plot_dir):
    os.makedirs(target_plot_dir)

In [ ]:
os.listdir(os.path.join(exp_root, exp_name))

In [ ]:
with open(os.path.join(exp_root, exp_name, f'NAMELIST_{exp_name}_atm')) as f:
    line = f.readline()
    in_output_nml = False
    in_ml_varlist = False
    
    nml_dict = {}
    
    while line:
        
        line = f.readline()
        if line.startswith('&output_nml'):
            in_output_nml = True
        elif line.strip() == '/':
            in_output_nml = False
            in_ml_varlist = False
        
        if in_output_nml and '=' in line:
            #print(line)
            splits = [s.strip() for s in line.split('=')]
            
            if splits[0] == 'output_filename':
                key = splits[1].replace('"', '')
                print(key)
                
                nml_dict[key] = []
                
            if splits[0] == 'ml_varlist':
                in_ml_varlist = True
                
        if in_ml_varlist:
            if line.strip().startswith('!'): # comment
                continue
   
            # read the comma separated list of output variables
            # account for first line starting with key
            splits = [s.split('=')[-1].strip().replace("'","") if '=' in s else s.replace("'","") for s in [s.strip() for s in line.split(',')]]
            clean_splits = [s for s in splits if len(s)>0]
            nml_dict[key].extend(clean_splits)

In [ ]:
pprint.pprint(nml_dict)

### Fieldmean plots of the vertically integrated variables

In [ ]:
n_cols = 5
total_plots = len(nml_dict[f'{exp_name}_atm_2d'])
n_rows = total_plots // n_cols

fig, ax = plt.subplots(n_rows, n_cols, figsize=(n_cols * 3.5, n_rows * 3.0))
ax = ax.flatten()

# define cdo infile
infile = os.path.join(exp_root, exp_name, out_atm_2d)

# obtain time steps for plotting
with xr.open_dataset(infile) as ds:
    ds = ds.load()
    
time_steps = ds['time'].values
time_steps -= time_steps[0]
time_steps *= 24 * 60 # time steps in minutes

for i, value in enumerate(nml_dict[f'{exp_name}_atm_2d']):
    x_fldmean = cdo.fldmean(input='-selname,' + value + ' ' + infile, returnXArray=value).squeeze()
    
    # get labels and units
    long_name = x_fldmean.attrs['long_name']
    units = x_fldmean.attrs['units']
    
    lbl = f'{value} ({units})'
    
    ax[i].plot(time_steps, x_fldmean)
    ax[i].set_xlabel('Time (minutes)')
    ax[i].set_ylabel(lbl)
    ax[i].set_title(long_name)
    
fig.tight_layout()
plt.savefig(os.path.join(target_plot_dir, f'{exp_name}_fldmean_atm_2d.png'), dpi=150, bbox_inches='tight')

### Fieldmean plots of the 3D variables

In [ ]:
n_cols = 5
total_plots = len(nml_dict[f'{exp_name}_atm_3d'])
n_rows = total_plots // n_cols

if total_plots % n_cols > 0:
    n_rows += 1

fig, ax = plt.subplots(n_rows, n_cols, figsize=(n_cols * 3.5, n_rows * 3.0))
ax = ax.flatten()

# define cdo infile
infile = os.path.join(exp_root, exp_name, out_atm_3d)

# obtain time steps for plotting
with xr.open_dataset(infile) as ds:
    ds = ds.load()
    
time_steps = ds['time'].values
time_steps -= time_steps[0]
time_steps *= 24 * 60 # time steps in minutes

for i, value in enumerate(nml_dict[f'{exp_name}_atm_3d']):
    x_fldmean = cdo.vertmean(input='-fldmean -selname,' + value + ' ' + infile, returnXArray=value).squeeze()
    
    if len(x_fldmean.shape) == 0:
        print(f'Skipping {value} of scalar dimension after averaging')
        continue
    
    # get labels and units
    long_name = x_fldmean.attrs['long_name']
    units = x_fldmean.attrs['units']
    
    lbl = f'{value} ({units})'
    
    ax[i].plot(time_steps, x_fldmean)
    ax[i].set_xlabel('Time (minutes)')
    ax[i].set_ylabel(lbl)
    ax[i].set_title(long_name)
    
fig.tight_layout()
plt.savefig(os.path.join(target_plot_dir, f'{exp_name}_vertmean_fldmean_atm_3d.png'), dpi=150, bbox_inches='tight')

## Comparison plots mean values

In [ ]:
def make_comparison_plot(experiments_to_compare=['warm_bubble_fortran', 'warm_bubble_cffi'], values='theta_v', colors=['C0', 'C3'], savefig=True):
    '''
    Compare field / vertically averaged value across several experiments
    
    Args:
        experiments_to_compare (list): experiment names
        values (string or list): parameter value(s) (need to be all atm_2d or atm_3d)
        colors (list): colors to use in the plot
        savefig (bool): if True, save the generated figure
    '''
    
    if isinstance(values, str):
        fig, ax = plt.subplots(1, 1, figsize=(3.5, 3.0))
        values = [values] # for compatibility
        ax = [ax]
        
    else:
        n_cols = min(6, len(values))
        total_plots = len(values)
        n_rows = total_plots // n_cols

        if total_plots % n_cols > 0:
            n_rows += 1

        fig, ax = plt.subplots(n_rows, n_cols, figsize=(n_cols * 3.5, n_rows * 3.0))
        ax = ax.flatten()
    
    # is the first value 2d or 3d
    is_atm_2d = True
    key3d = [kk for kk in list(nml_dict.keys()) if kk.endswith('atm_3d')]
    if values[0] in nml_dict[key3d[0]]:
        is_atm_2d = False
        
    if is_atm_2d: # fieldmean
        for exp_name, lc in zip(experiments_to_compare, colors):
            infile = os.path.join(exp_root, exp_name, f'{exp_name}_atm_2d_ml_20080801T000000Z.nc')
        
            # obtain time steps for plotting
            with xr.open_dataset(infile) as ds:
                ds = ds.load()
    
            time_steps = ds['time'].values
            time_steps -= time_steps[0]
            time_steps *= 24 * 60 # time steps in minutes

            for i, value in enumerate(values):
                x_fldmean = cdo.fldmean(input='-selname,' + value + ' ' + infile, returnXArray=value).squeeze()
    
                if len(x_fldmean.shape) == 0:
                    print(f'Skipping {value} of scalar dimension after averaging')
                    continue
    
                ax[i].plot(time_steps, x_fldmean, lw=2, label=exp_name, color=lc)
        
                # get labels and units
                long_name = x_fldmean.attrs['long_name']
                units = x_fldmean.attrs['units']
    
                lbl = f'{value} ({units})'
    
                ax[i].set_xlabel('Time (minutes)')
                ax[i].set_ylabel(lbl)
                ax[i].set_title(long_name)
                ax[i].legend()
    
    else: # vertmean(fieldmean)    
        for exp_name, lc in zip(experiments_to_compare, colors):
            infile = os.path.join(exp_root, exp_name, f'{exp_name}_atm_3d_ml_20080801T000000Z.nc')
            
            # obtain time steps for plotting
            with xr.open_dataset(infile) as ds:
                ds = ds.load()
    
            time_steps = ds['time'].values
            time_steps -= time_steps[0]
            time_steps *= 24 * 60 # time steps in minutes

            for i, value in enumerate(values):
                x_fldmean = cdo.vertmean(input='-fldmean -selname,' + value + ' ' + infile, returnXArray=value).squeeze()
    
                ax[i].plot(time_steps, x_fldmean, lw=2, label=exp_name, color=lc)
                
                # get labels and units
                long_name = x_fldmean.attrs['long_name']
                units = x_fldmean.attrs['units']
    
                lbl = f'{value} ({units})'
    
                ax[i].set_xlabel('Time (minutes)')
                ax[i].set_ylabel(lbl)
                ax[i].set_title(long_name)
                ax[i].legend()
        

    fig.tight_layout()
        
    if savefig:
        target = 'compare'
        for exp_name in experiments_to_compare:
            target += f'_{exp_name}'
            
        for value in values:
            target += f'_{value}'
        
        plt.savefig(os.path.join(target_plot_dir, f'{target}.png'), dpi=150, bbox_inches='tight')

In [ ]:
experiments_to_compare=['cold_bubble_fortran', 'cold_bubble_cffi']

In [ ]:
make_comparison_plot(experiments_to_compare=experiments_to_compare, values='theta_v', savefig=True)

In [ ]:
make_comparison_plot(experiments_to_compare=experiments_to_compare, values=['ua', 'va'], savefig=True)

In [ ]:
make_comparison_plot(experiments_to_compare=experiments_to_compare, values=['clw', 'cli', 'qr', 'qs', 'qg', 'qh'])

In [ ]:
make_comparison_plot(experiments_to_compare=experiments_to_compare, values=['qnc', 'qni', 'qnr', 'qns', 'qng', 'qnh'])

## Vertical plots at different time stamps

In [ ]:
def make_vertical_comparison_plot(experiments_to_compare=['warm_bubble_fortran', 'warm_bubble_cffi'], 
                                  value='theta_v', 
                                  colors=['C0', 'C3'], 
                                  times=['00:00:00', '00:20:00', '00:40:00', '01:00:00', '01:20:00', '01:40:00'], 
                                  max_height=7000,
                                  savefig=True):
    '''
    Compare field averaged value across several experiments
    Plots the vertical profile for the given time stamps
    
    Args:
        experiments_to_compare (list): experiment names
        value (string): parameter value (from atm_3d)
        colors (list): colors to use in the plot
        times (list): timestamps to use
        max_height (float): maximum geometric height to display
        savefig (bool): if True, save the generated figure
    '''
    
    n_cols = len(experiments_to_compare)
    n_rows = 1 #total_plots // n_cols

    fig, ax = plt.subplots(n_rows, n_cols, figsize=(n_cols * 3.5, n_rows * 4.0))
    ax = ax.flatten()
    
    # is the first value 2d or 3d
    is_atm_2d = True
    key3d = [kk for kk in list(nml_dict.keys()) if kk.endswith('atm_3d')]
    if value in nml_dict[key3d[0]]:
        is_atm_2d = False
        
    if is_atm_2d: # not applicable
        return
    
    for i, exp_name in enumerate(experiments_to_compare):
        infile = os.path.join(exp_root, exp_name, f'{exp_name}_atm_3d_ml_20080801T000000Z.nc')
        
        zg = cdo.fldmean(input='-selname,' + 'zg' + ' ' + infile, returnXArray='zg').squeeze().values
        
        for time in times:
            t_fldmean = cdo.seltime(time, input='-fldmean -selname,' + value + ' ' + infile, returnXArray=value).squeeze()
            ax[i].plot(t_fldmean, zg, label=time)
                
        # get labels and units
        long_name = t_fldmean.attrs['long_name']
        units = t_fldmean.attrs['units']
    
        lbl = f'{long_name} ({units})'
    
        ax[i].set_xlabel(lbl)
        ax[i].set_ylabel('Geometric height (m)')
        ax[i].set_title(exp_name)
        ax[i].legend()
        ax[i].set_ylim(0, max_height)

    fig.tight_layout()
        
    if savefig:
        target = 'compare'
        for exp_name in experiments_to_compare:
            target += f'_{exp_name}'
        
        plt.savefig(os.path.join(target_plot_dir, f'{target}_time_series_{value}.png'), dpi=150, bbox_inches='tight')

In [ ]:
make_vertical_comparison_plot(value='hus', experiments_to_compare=['warm_bubble_fortran', 'warm_bubble_cffi'])

In [ ]:
make_vertical_comparison_plot(value='ta', experiments_to_compare=['warm_bubble_xy_fortran', 'warm_bubble_xy_cffi'])

In [ ]:
make_vertical_comparison_plot(value='ta', experiments_to_compare=['warm_bubble_fortran', 'warm_bubble_1mom_fortran'])

In [ ]:
make_vertical_comparison_plot(value='ta', experiments_to_compare=['warm_bubble_fortran', 'warm_bubble_cffi', 'cold_bubble_fortran', 'cold_bubble_cffi'])

In [ ]:
make_vertical_comparison_plot(value='theta_v', experiments_to_compare=['warm_bubble_fortran', 'warm_bubble_cffi', 'cold_bubble_fortran', 'cold_bubble_cffi'])

In [ ]:
make_vertical_comparison_plot(value='hus', experiments_to_compare=['warm_bubble_fortran', 'warm_bubble_cffi', 'cold_bubble_fortran', 'cold_bubble_cffi'])

In [ ]:
make_vertical_comparison_plot(value='cli', experiments_to_compare=['warm_bubble_fortran', 'warm_bubble_cffi', 'cold_bubble_fortran', 'cold_bubble_cffi'])

In [ ]:
make_vertical_comparison_plot(value='qg', experiments_to_compare=['warm_bubble_fortran', 'warm_bubble_cffi', 'cold_bubble_fortran', 'cold_bubble_cffi'])

In [ ]:
make_vertical_comparison_plot(value='qs', experiments_to_compare=['warm_bubble_fortran', 'warm_bubble_cffi', 'cold_bubble_fortran', 'cold_bubble_cffi'])

In [ ]:
make_vertical_comparison_plot(value='clw', experiments_to_compare=['warm_bubble_fortran', 'warm_bubble_cffi', 'cold_bubble_fortran', 'cold_bubble_cffi'])

In [ ]:
make_vertical_comparison_plot(value='qr', experiments_to_compare=['warm_bubble_fortran', 'warm_bubble_cffi', 'cold_bubble_fortran', 'cold_bubble_cffi'])

## Bubble

In [ ]:
with xr.open_dataset(infile) as tmp:
    ta = tmp['ta']
    tmp.close()

In [ ]:
plt.scatter(ta.clon, ta.clat, c=ta[0, -1])
plt.colorbar()

## Moments only

In [ ]:
def scatter_moments(experiments_to_compare=['warm_bubble_fortran', 'warm_bubble_cffi'],
                    value='qr',
                    times=['00:00:00', '00:20:00', '00:40:00', '01:00:00', '01:20:00', '01:40:00'], 
                    avg='vertmean',
                    savefig=True
                   ):
    '''
    
    
    '''
    n_cols = len(times)
    n_rows = 1 #total_plots // n_cols

    fig, ax = plt.subplots(n_rows, n_cols, figsize=(n_cols * 3.5, n_rows * 4.0), sharex=True, sharey=True)
    ax = ax.flatten()
    [axx.set_aspect('equal') for axx in ax]
    
    # is the first value 2d or 3d
    is_atm_2d = True
    key3d = [kk for kk in list(nml_dict.keys()) if kk.endswith('atm_3d')]
    if value in nml_dict[key3d[0]]:
        is_atm_2d = False
        
    if is_atm_2d: # not applicable
        return
    
    for i, time in enumerate(times):
        #if time != '00:40:00':
        #    continue
        print(time)
            
        t_vals = []
        
        for j, exp_name in enumerate(experiments_to_compare):
            infile = os.path.join(exp_root, exp_name, f'{exp_name}_atm_3d_ml_20080801T000000Z.nc')
        
            t_val = cdo.seltime(time, input='-' + avg + ' -selname,' + value + ' ' + infile, returnXArray=value).squeeze()
            t_vals.append(t_val.values)
            
        ax[i].scatter(t_vals[0], t_vals[1])
                            
        ax[i].set_xlabel(f'{value} ({experiments_to_compare[0]})')
        ax[i].set_ylabel(f'{value} ({experiments_to_compare[1]})')
        ax[i].set_title(time)

    fig.tight_layout()
        
    if savefig:
        target = 'compare'
        for exp_name in experiments_to_compare:
            target += f'_{exp_name}'
        
        plt.savefig(os.path.join(target_plot_dir, f'{target}_scatter_{value}_{avg}.png'), dpi=150, bbox_inches='tight')
    

In [ ]:
scatter_moments(value='qnr')

In [ ]:
def hovmoller_time(experiments_to_compare=['warm_bubble_fortran', 'warm_bubble_cffi'],
                    value='qr',
                    max_height=10000,
                    cmap='viridis',
                    same_cmap=True,
                    savefig=True
                   ):
    '''
    
    '''
    n_cols = len(experiments_to_compare)
    n_rows = 1 #total_plots // n_cols

    fig, ax = plt.subplots(n_rows, n_cols, figsize=(n_cols * 4.5, n_rows * 4.0), sharex=True, sharey=True)
    ax = ax.flatten()
    [axx.set_aspect('equal') for axx in ax]
    
    # is the first value 2d or 3d
    is_atm_2d = True
    key3d = [kk for kk in list(nml_dict.keys()) if kk.endswith('atm_3d')]
    if value in nml_dict[key3d[0]]:
        is_atm_2d = False
        
    if is_atm_2d: # not applicable
        return
    
    all_tvals = []
    
    for i, exp_name in enumerate(experiments_to_compare):
        infile = os.path.join(exp_root, exp_name, f'{exp_name}_atm_3d_ml_20080801T000000Z.nc')
        
        zg = cdo.fldmean(input='-selname,' + 'zg' + ' ' + infile, returnXArray='zg').squeeze().values
        t_val = cdo.fldmean(input='-selname,' + value + ' ' + infile, returnXArray=value).squeeze()
        all_tvals.append(t_val)
        
    if same_cmap:
        vmin = np.min( all_tvals )
        vmax = np.max( all_tvals )
    else:
        vmin = None
        vmax = None
        
    for i, exp_name in enumerate(experiments_to_compare):
        
        t0  = 0
        t1 = 120
        img = ax[i].imshow(all_tvals[i].T, aspect='auto', extent=[t0, t1, zg[-1], zg[0]], cmap=cmap, vmin=vmin, vmax=vmax)
        ax[i].set_ylim(0, max_height)
                            
        ax[i].set_xlabel('Time (minutes)')
        ax[i].set_ylabel('Geometric height (m)')
        ax[i].set_title(exp_name)
        plt.colorbar(img, ax=ax[i], label=value)

    fig.tight_layout()
        
    if savefig:
        target = 'compare'
        for exp_name in experiments_to_compare:
            target += f'_{exp_name}'
        
        plt.savefig(os.path.join(target_plot_dir, f'{target}_hovmoller_time_{value}.png'), dpi=150, bbox_inches='tight')
    

In [ ]:
hovmoller_time(value='clw')

In [ ]:
hovmoller_time(value='qr')

In [ ]:
hovmoller_time(value='qs', experiments_to_compare=['cold_bubble_fortran', 'cold_bubble_cffi'])

In [ ]:
hovmoller_time(value='qnr')

In [ ]:
hovmoller_time(value='qnc')

In [ ]:
hovmoller_time(value='ta', experiments_to_compare=['cold_bubble_fortran', 'cold_bubble_cffi'])

In [ ]:
hovmoller_time(value='ua', cmap='RdBu')

In [ ]:
hovmoller_time(value='va', cmap='RdBu')

## Regridding

Define a latlon grid. Call to cdo for interpolation weights.

In [ ]:
torus = xr.open_dataset(f'{exp_root}/{exp_name}/Torus_Triangles_20x22_5000m.nc').load()
torus

In [ ]:
np.rad2deg(-0.164) * 2 / 22

In [ ]:
with open(f'{exp_root}/{exp_name}/remap_griddes.txt', 'w') as f:
    f.write(
'''
# gridID 1
#
gridtype  = lonlat
gridsize  = 440
xsize     = 20
ysize     = 22
xname     = x
xlongname = "x_zonal"
xunits    = "degrees_east"
yname     = y
ylongname = "y_meridional"
yunits    = "degrees_north"
xfirst    = -179.95
xinc      = 18
yfirst    = -9.5
yinc      = 0.85
'''
    )

In [ ]:
%%time
cdo.gendis(f'{exp_root}/{exp_name}/remap_griddes.txt', input=f'-setgrid,{exp_root}/{exp_name}/Torus_Triangles_20x22_5000m.nc {exp_root}/{exp_name}/{exp_name}_atm_2d_ml_20080801T000000Z.nc',
           output=f'{exp_root}/{exp_name}/weightfile_{exp_name}.nc')

Remap the experiment files (saved in orignal experiment directory)
- expfile.nc --> remap_expfile.nc

In [ ]:
%%time
cdo.remap(f'{exp_root}/{exp_name}/remap_griddes.txt,{exp_root}/{exp_name}/weightfile_{exp_name}.nc',
          input=f'{exp_root}/{exp_name}/{exp_name}_atm_2d_ml_20080801T000000Z.nc',
          output=f'{exp_root}/{exp_name}/remap_{exp_name}_atm_2d_ml_20080801T000000Z.nc')

In [ ]:
%%time
cdo.remap(f'{exp_root}/{exp_name}/remap_griddes.txt,{exp_root}/{exp_name}/weightfile_{exp_name}.nc',
          input=f'{exp_root}/{exp_name}/{exp_name}_atm_3d_ml_20080801T000000Z.nc',
          output=f'{exp_root}/{exp_name}/remap_{exp_name}_atm_3d_ml_20080801T000000Z.nc')

In [ ]:
with xr.open_dataset(f'{exp_root}/{exp_name}/remap_{exp_name}_atm_3d_ml_20080801T000000Z.nc') as tmp:
    test = tmp.load()
test

In [ ]:
plt.imshow(test['ta'][0][-1])

In [ ]:
def hovmoller_diagrams(value,
                       max_height=10000,
                       cmap='viridis',
                       share_cmap=True,
                       use_scn=True,
                       experiments_to_compare=['warm_bubble_fortran', 'warm_bubble_cffi'],
                       symmetric_cmap=False):
    '''
    Generate Hovmoller diagrams of regridded quantities
    
    Args:
      value (str): value to plot
      max_height (int, float): maximum geometric height
      cmap (str): matplotlib colormap
      share_cmap (bool): if True, share colormap between all axes
      use_scn (bool): if True, use scientific notation in colorbar ticklabels
      experiments_to_compare (list): experiments to compare (one row = one experiment)
      symmetric_cmap (bool): if True, use symmetric vmin/vmax on colormaps (for values that center around 0)
    
    '''
    
    if share_cmap and len(experiments_to_compare)>1:
        raise ValueError("Do not share cmap for more than one experiment")
    
    if use_scn:
        formatter = ticker.ScalarFormatter(useMathText=True)
        formatter.set_scientific(True) 
        formatter.set_powerlimits((-1,1))
    else:
        formatter = lambda x, _: f"{x:.0f}"
        
    fig, ax = plt.subplots(len(experiments_to_compare), 3, sharex=True, figsize=(4*3, len(experiments_to_compare) * 4.5))
    
    if share_cmap:
        v_zonal = cdo.zonmean(input=f'-vertmean -selname,{value} {exp_root}/{exp_name}/remap_{exp_name}_atm_3d_ml_20080801T000000Z.nc', returnXArray=value).squeeze()
        v_meridional = cdo.mermean(input=f'-vertmean -selname,{value} {exp_root}/{exp_name}/remap_{exp_name}_atm_3d_ml_20080801T000000Z.nc', returnXArray=value).squeeze()
        v_vertical = cdo.fldmean(input=f'-selname,{value} {exp_root}/{exp_name}/remap_{exp_name}_atm_3d_ml_20080801T000000Z.nc', returnXArray=value).squeeze()
    
        # helper
        zg = cdo.fldmean(input='-selname,' + 'zg' + ' ' + f'{exp_root}/{exp_name}/remap_{exp_name}_atm_3d_ml_20080801T000000Z.nc', returnXArray='zg').squeeze().values
        time_steps = v_zonal['time'].values
        time_steps -= time_steps[0]
        time_steps *= 24 * 60 # time steps in minutes
    
        vmin = min([v_meridional.min(), v_zonal.min(), v_vertical.min()])
        vmax = max([v_meridional.max(), v_zonal.max(), v_vertical.max()])

        if symmetric_cmap:
            vmin = -np.max(np.abs([vmin, vmax]))
            vmax = -vmin
    
        ax[0].contourf(v_zonal.T, vmin=vmin, vmax=vmax, cmap=cmap)
        ax[1].contourf(v_meridional.T, vmin=vmin, vmax=vmax, cmap=cmap)
        img = ax[2].contourf(v_vertical.T, vmin=vmin, vmax=vmax, cmap=cmap)
        plt.colorbar(img, ax=ax, label=value, format=formatter)
        
        ax[0].set_xlabel('Time (mins)')
        ax[1].set_xlabel('Time (mins)')
        ax[2].set_xlabel('Time (mins)')
        ax[0].set_ylabel('x direction')
        ax[1].set_ylabel('y direction')
        ax[2].set_ylabel('Geometric height')
    
        ax[2].set_ylim(0, max_height)
    
    else:
        
        for i, exp_name in enumerate(experiments_to_compare):
            if not os.path.exists(f'{exp_root}/{exp_name}/weightfile_{exp_name}.nc'):
                with open(f'{exp_root}/{exp_name}/remap_griddes.txt', 'w') as f:
                    f.write(
'''
# gridID 1
#
gridtype  = lonlat
gridsize  = 440
xsize     = 20
ysize     = 22
xname     = x
xlongname = "x_zonal"
xunits    = "degrees_east"
yname     = y
ylongname = "y_meridional"
yunits    = "degrees_north"
xfirst    = -179.95
xinc      = 18
yfirst    = -9.5
yinc      = 0.85
'''
                    )
                cdo.gendis(f'{exp_root}/{exp_name}/remap_griddes.txt', input=f'-setgrid,{exp_root}/{exp_name}/Torus_Triangles_20x22_5000m.nc {exp_root}/{exp_name}/{exp_name}_atm_2d_ml_20080801T000000Z.nc',
                           output=f'{exp_root}/{exp_name}/weightfile_{exp_name}.nc')
            
            if not os.path.exists(f'{exp_root}/{exp_name}/remap_{exp_name}_atm_3d_ml_20080801T000000Z.nc'):
                cdo.remap(f'{exp_root}/{exp_name}/remap_griddes.txt,{exp_root}/{exp_name}/weightfile_{exp_name}.nc',
                          input=f'{exp_root}/{exp_name}/{exp_name}_atm_3d_ml_20080801T000000Z.nc',
                          output=f'{exp_root}/{exp_name}/remap_{exp_name}_atm_3d_ml_20080801T000000Z.nc')
            
            v_zonal = cdo.zonmean(input=f'-vertmean -selname,{value} {exp_root}/{exp_name}/remap_{exp_name}_atm_3d_ml_20080801T000000Z.nc', returnXArray=value).squeeze()
            v_meridional = cdo.mermean(input=f'-vertmean -selname,{value} {exp_root}/{exp_name}/remap_{exp_name}_atm_3d_ml_20080801T000000Z.nc', returnXArray=value).squeeze()
            v_vertical = cdo.fldmean(input=f'-selname,{value} {exp_root}/{exp_name}/remap_{exp_name}_atm_3d_ml_20080801T000000Z.nc', returnXArray=value).squeeze()
        
            # helper
            zg = cdo.fldmean(input='-selname,' + 'zg' + ' ' + f'{exp_root}/{exp_name}/remap_{exp_name}_atm_3d_ml_20080801T000000Z.nc', returnXArray='zg').squeeze().values
            time_steps = v_zonal['time'].values
            time_steps -= time_steps[0]
            time_steps *= 24 * 60 # time steps in minutes
        
            img = ax[i, 0].contourf(time_steps, np.arange(22), v_zonal.T, cmap=cmap)
            plt.colorbar(img, ax=ax[i, 0], label=value, location='bottom', format=formatter)
            img = ax[i, 1].contourf(time_steps, np.arange(20), v_meridional.T, cmap=cmap)
            plt.colorbar(img, ax=ax[i, 1], label=value, location='bottom', format=formatter)
            img = ax[i, 2].contourf(time_steps, zg, v_vertical.T, cmap=cmap)
            plt.colorbar(img, ax=ax[i, 2], label=value, location='bottom', format=formatter)
    
            ax[i, 0].set_xlabel('Time (mins)')
            ax[i, 1].set_xlabel('Time (mins)')
            ax[i, 2].set_xlabel('Time (mins)')
            ax[i, 0].set_ylabel('x direction')
            ax[i, 1].set_ylabel('y direction')
            ax[i, 2].set_ylabel('Geometric height')
    
            ax[i, 2].set_ylim(0, max_height)

    
        fig.tight_layout()
    
    plt.savefig(f'./analysis_plots/hovmoller_{exp_name}_{value}.png', dpi=150, bbox_inches='tight')
    

In [ ]:
hovmoller_diagrams('ua', cmap='Spectral', symmetric_cmap=True, share_cmap=True)

In [ ]:
hovmoller_diagrams('va', cmap='Spectral', symmetric_cmap=True, share_cmap=False)

In [ ]:
hovmoller_diagrams('qr', cmap='Blues', symmetric_cmap=False, share_cmap=False, experiments_to_compare=['warm_bubble_fortran', 'warm_bubble_cffi', 'cold_bubble_fortran', 'cold_bubble_cffi'])

In [ ]:
hovmoller_diagrams('qnr', cmap='Blues', symmetric_cmap=False, share_cmap=False, experiments_to_compare=['warm_bubble_fortran', 'warm_bubble_cffi', 'cold_bubble_fortran', 'cold_bubble_cffi'])

In [ ]:
hovmoller_diagrams('cli', cmap='Blues', symmetric_cmap=False, share_cmap=False, experiments_to_compare=['warm_bubble_fortran', 'warm_bubble_cffi', 'cold_bubble_fortran', 'cold_bubble_cffi'])

In [ ]:
hovmoller_diagrams('qni', cmap='Blues', symmetric_cmap=False, share_cmap=False, experiments_to_compare=['warm_bubble_fortran', 'warm_bubble_cffi', 'cold_bubble_fortran', 'cold_bubble_cffi'])

In [ ]:
hovmoller_diagrams('qs', cmap='Blues', symmetric_cmap=False, share_cmap=False, experiments_to_compare=['warm_bubble_fortran', 'warm_bubble_cffi', 'cold_bubble_fortran', 'cold_bubble_cffi'])

In [ ]:
hovmoller_diagrams('qns', cmap='Blues', symmetric_cmap=False, share_cmap=False, experiments_to_compare=['warm_bubble_fortran', 'warm_bubble_cffi', 'cold_bubble_fortran', 'cold_bubble_cffi'])

In [ ]:
hovmoller_diagrams('ta', cmap='magma', symmetric_cmap=False, share_cmap=False, use_scn=False)

In [ ]:
hovmoller_diagrams('theta_v', cmap='magma', symmetric_cmap=False, share_cmap=False, use_scn=False)

## Individual moments

In [ ]:
import re

In [ ]:
log_fortran = os.path.join(exp_root, 'special_logs', 'log_warm_bubble_fortran_DEBUG')
log_cffi = os.path.join(exp_root, 'special_logs', 'log_warm_bubble_cffi_DEBUG')

First time stamp with non zero input moments for the warm bubble scenario:
    
    time_stamp = '2008-08-01 00:13:20.000'

In [ ]:
time_stamp = '2008-08-01 00:28:00.000'
df_columns = ['time_stamp', 'i', 'k', 'is_mom_in', 'qc', 'nc', 'qr', 'nr']

In [ ]:
def read_moments_from_log(logfile, df_columns):
    
    dict_mom = {}
    for c in df_columns:
        dict_mom[c] = []

    with open(logfile) as f:
        line = f.readline()
        is_print = False
        in_time_step = False
    
        while line:
        
            if is_print and 'two moment mcrph ends' in line:
                print(line)
                break # stop at the next time step
            if time_stamp in line:
                in_time_step = True
                print(line)
            
            if in_time_step and 'mo_2mom_mcrph_driver: calling clouds_twomoment' in line:
                is_print = True
                print(line)
            
            if is_print and 'mo_2mom_mcrph_main' in line:
                moms = [float(x) for x in  re.findall(r'\d+\.\d+E[+-]\d+', line)]
                if len(moms) == 4 and not np.allclose(moms, 0, atol=1e-20):
                    ik = re.findall(r'\(i,\s*k\)\s*=\s*\d+\s*\d+', line)[0]
                    p = re.compile(r'\d+')
                    i, k = [int(x) for x in p.findall(ik)]
                
                    dict_mom['time_stamp'].append(time_stamp)
                    dict_mom['is_mom_in'].append('m_in' in line)
                    dict_mom['i'].append(i)
                    dict_mom['k'].append(k)
                    dict_mom['qc'].append(moms[0])
                    dict_mom['nc'].append(moms[1])
                    dict_mom['qr'].append(moms[2])
                    dict_mom['nr'].append(moms[3])
        
            line = f.readline()
            
        df_mom = pd.DataFrame(dict_mom)
        df_mom['experiment'] = os.path.basename(logfile)
        df_mom['exp_root'] = os.path.dirname(logfile)
        return df_mom

In [ ]:
%%time
df_mom_fortran = read_moments_from_log(log_fortran, df_columns)

In [ ]:
%%time
df_mom_cffi = read_moments_from_log(log_cffi, df_columns)

In [ ]:
df_combined = pd.concat([df_mom_fortran, df_mom_cffi])
df_combined.describe()

In [ ]:
df_combined.groupby(['experiment'])['is_mom_in'].mean()

Where do we encounter non-zero moments?

In [ ]:
fig, ax = plt.subplots(1, 2, sharex=True, sharey=True, figsize=(8, 4))
sns.scatterplot(data=df_mom_fortran.loc[df_mom_fortran['is_mom_in']], x='i', y='k', ax=ax[0], alpha=0.2)
sns.scatterplot(data=df_mom_cffi.loc[df_mom_cffi['is_mom_in']], x='i', y='k', ax=ax[1], color='C1', alpha=0.2)
ax[0].set_title(df_mom_fortran['experiment'][0])
ax[1].set_title(df_mom_cffi['experiment'][0])
fig.tight_layout()

In [ ]:
fig, ax = plt.subplots(1, 4, figsize=(20, 4), sharex=True)
ax=ax.flatten()

sns.barplot(data=df_combined, x='is_mom_in', y='qc', hue='experiment', ax=ax[0])
sns.barplot(data=df_combined, x='is_mom_in', y='qr', hue='experiment', ax=ax[1])
sns.barplot(data=df_combined, x='is_mom_in', y='nc', hue='experiment', ax=ax[2])
sns.barplot(data=df_combined, x='is_mom_in', y='nr', hue='experiment', ax=ax[3])

ax[0].set_xticklabels(['out', 'in']) # keep order!
ax[0].invert_xaxis()
[axx.set_xlabel('Moment') for axx in ax]

fig.tight_layout()

How much do the moments change during the warm rain process?

In [ ]:
fig, ax = plt.subplots(2, 4, figsize=(18, 8))

# fortran
ax[0, 0].scatter(df_mom_fortran.loc[df_mom_fortran['is_mom_in']]['qc'], 
                 df_mom_fortran.loc[~df_mom_fortran['is_mom_in']]['qc'])
ax[0, 0].set_title('qc (fortran)')
ax[0, 1].scatter(df_mom_fortran.loc[df_mom_fortran['is_mom_in']]['nc'], 
                 df_mom_fortran.loc[~df_mom_fortran['is_mom_in']]['nc'])
ax[0, 1].set_title('nc (fortran)')
ax[0, 2].scatter(df_mom_fortran.loc[df_mom_fortran['is_mom_in']]['qr'], 
                 df_mom_fortran.loc[~df_mom_fortran['is_mom_in']]['qr'])
ax[0, 2].set_title('qr (fortran)')
ax[0, 3].scatter(df_mom_fortran.loc[df_mom_fortran['is_mom_in']]['nr'], 
                 df_mom_fortran.loc[~df_mom_fortran['is_mom_in']]['nr'])
ax[0, 3].set_title('nr (fortran)')

# cffi
ax[1, 0].scatter(df_mom_cffi.loc[df_mom_cffi['is_mom_in']]['qc'], 
                 df_mom_cffi.loc[~df_mom_cffi['is_mom_in']]['qc'])
ax[1, 0].set_title('qc (cffi)')
ax[1, 1].scatter(df_mom_cffi.loc[df_mom_cffi['is_mom_in']]['nc'], 
                 df_mom_cffi.loc[~df_mom_cffi['is_mom_in']]['nc'])
ax[1, 1].set_title('nc (cffi)')
ax[1, 2].scatter(df_mom_cffi.loc[df_mom_cffi['is_mom_in']]['qr'], 
                 df_mom_cffi.loc[~df_mom_cffi['is_mom_in']]['qr'])
ax[1, 2].set_title('qr (cffi)')
ax[1, 3].scatter(df_mom_cffi.loc[df_mom_cffi['is_mom_in']]['nr'], 
                 df_mom_cffi.loc[~df_mom_cffi['is_mom_in']]['nr'])
ax[1, 3].set_title('nr (cffi)')

[axx.set_xscale('log') for axx in ax.flatten()]
[axx.set_yscale('log') for axx in ax.flatten()]
[axx.set_aspect('equal') for axx in ax.flatten()]
[axx.set_xlabel('Moments in') for axx in ax.flatten()]
[axx.set_ylabel('Moments out') for axx in ax.flatten()]

for axx in ax.flatten():
    x0, x1 = axx.get_xlim()
    axx.plot(np.linspace(x0, x1), np.linspace(x0, x1), color='red')

fig.tight_layout()

plt.show()

In [ ]:
## debug

In [ ]:
os.listdir('/

In [ ]:
ds = xr.open_dataset('/home/k/k202141/rootgit/icon-aes/experiments/warm_bubble_fortran/warm_bubble_fortran_atm_2d_ml_20080801T000000Z.nc')

In [ ]:
ds

In [ ]:
tmpf = cdo.seltime('00:30:00', input='-fldmean -selname,' + 'qnr' + ' ' + '/home/k/k202141/rootgit/icon-aes/experiments/warm_bubble_fortran/warm_bubble_fortran_atm_3d_ml_20080801T000000Z.nc', returnXArray='qnr').squeeze()

In [ ]:
tmpf2 = cdo.seltime('00:30:00', input='-fldmean -selname,' + 'qr' + ' ' + \
                    '/home/k/k202141/rootgit/icon-aes/experiments/warm_bubble_fortran/warm_bubble_fortran_atm_3d_ml_20080801T000000Z.nc', returnXArray='qr').squeeze()

In [ ]:
tmpc = cdo.seltime('00:30:00', input='-fldmean -selname,' + 'qnr' + ' ' + '/home/k/k202141/rootgit/icon-aes/experiments/warm_bubble_cffi/warm_bubble_cffi_atm_3d_ml_20080801T000000Z.nc', returnXArray='qnr').squeeze()

In [ ]:
tmpf

In [ ]:
tmpf2

In [ ]:
tmpc

In [ ]:
fig, axs = plt.subplots(1, 4, sharex=True, sharey=True, figsize=(20, 5))
axs = axs.flatten()

zg = cdo.fldmean(input='-selname,' + 'zg' + ' ' + '/home/k/k202141/rootgit/icon-aes/experiments/warm_bubble_fortran/warm_bubble_fortran_atm_3d_ml_20080801T000000Z.nc', returnXArray='zg').squeeze().values

for ax, time in zip(axs, ['00:20:00', '00:25:00', '00:30:00', '00:35:00']):
    
    tmpf = cdo.seltime(time, input='-fldmean -selname,' + 'qnr' + ' ' + '/home/k/k202141/rootgit/icon-aes/experiments/warm_bubble_fortran/warm_bubble_fortran_atm_3d_ml_20080801T000000Z.nc', returnXArray='qnr').squeeze()    
    tmpc = cdo.seltime(time, input='-fldmean -selname,' + 'qnr' + ' ' + '/home/k/k202141/rootgit/icon-aes/experiments/warm_bubble_cffi/warm_bubble_cffi_atm_3d_ml_20080801T000000Z.nc', returnXArray='qnr').squeeze()    
    
    ax.plot(zg, tmpf.values, label='bulk moment')
    ax.plot(zg, tmpc.values, label='superdropnet', color='C3')
    ax.legend()
    ax.set_title(time)
    ax.set_xlabel('Geometric height (m)')
    
axs[0].set_xlim(0, 6000)
axs[0].set_ylabel(tmpf.attrs['long_name'])

In [ ]:
tmpf